### 12306模拟登录
- url:https://kyfw.12306.cn/otn/login/init
- 分析：主要是对验证码的处理
    - 基于selenium实现。
    - 需要使用selenium将登录页面打开
    - 我们即将实现的模拟登录页面和验证码图片一定是一一匹配
        - 验证码每次请求都会发生变化
    - 如何保证我们捕获的验证码和当次登录是一一匹配？
        - 将当前selenium打开的登录页面中的验证码图片裁剪下来即可。

In [7]:
#!/usr/bin/env python
# coding:utf-8

import requests
from hashlib import md5

class Chaojiying_Client(object):

    def __init__(self, username, password, soft_id):
        self.username = username
        password =  password.encode('utf8')
        self.password = md5(password).hexdigest()
        self.soft_id = soft_id
        self.base_params = {
            'user': self.username,
            'pass2': self.password,
            'softid': self.soft_id,
        }
        self.headers = {
            'Connection': 'Keep-Alive',
            'User-Agent': 'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0)',
        }

    def PostPic(self, im, codetype):
        """
        im: 图片字节
        codetype: 题目类型 参考 http://www.chaojiying.com/price.html
        """
        params = {
            'codetype': codetype,
        }
        params.update(self.base_params)
        files = {'userfile': ('ccc.jpg', im)}
        r = requests.post('http://upload.chaojiying.net/Upload/Processing.php', data=params, files=files, headers=self.headers)
        return r.json()

    def ReportError(self, im_id):
        """
        im_id:报错题目的图片ID
        """
        params = {
            'id': im_id,
        }
        params.update(self.base_params)
        r = requests.post('http://upload.chaojiying.net/Upload/ReportError.php', data=params, headers=self.headers)
        return r.json()


In [8]:
def tranformImgCode(imgPath,imgType):
    chaojiying = Chaojiying_Client('bobo328410948', 'bobo328410948', '899370')
    im = open(imgPath, 'rb').read()
    return chaojiying.PostPic(im,imgType)['pic_str']

In [5]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from time import sleep
from PIL import Image

In [14]:
bro = webdriver.Chrome(executable_path='./chromedriver')
bro.get('https://kyfw.12306.cn/otn/login/init')
sleep(2)
#将当次登录对应的验证码图片进行裁剪
bro.save_screenshot('main.png')#当前登录页面对应的完整图片
#定位到了验证码图片对应的标签
img_tag = bro.find_element_by_xpath('//*[@id="loginForm"]/div/ul[2]/li[4]/div/div/div[3]/img')
#裁剪出验证码图片
location = img_tag.location#img_tag表示的图片在当前页面中左下角坐标
size = img_tag.size #验证码图片的尺寸
#基于location和size制定出裁剪的范围
rangle = (int(location['x']),int(location['y']),int(location['x']+size['width']),int(location['y']+size['height']))
#根据rangle表示的裁剪范围进行图片的裁剪
#基于图片进行裁剪：pip install PIL/Pillow
i = Image.open('./main.png')
frame = i.crop(rangle)
frame.save('./code.png')
#识别验证码图片
#result是返回需要点击的坐标
result = tranformImgCode('./code.png',9004)
#61,71|118,137==>[[61,71],[118,137]]
all_list = []#[[61,71],[118,137]]
if '|' in result:
    list_1 = result.split('|')
    count_1 = len(list_1)
    for i in range(count_1):
        xy_list = []
        x = int(list_1[i].split(',')[0])
        y = int(list_1[i].split(',')[1])
        xy_list.append(x)
        xy_list.append(y)
        all_list.append(xy_list)
else:
    x = int(result.split(',')[0])
    y = int(result.split(',')[1])
    xy_list = []
    xy_list.append(x)
    xy_list.append(y)
    all_list.append(xy_list)
for xy in all_list:
    x = xy[0]
    y = xy[1]
    ActionChains(bro).move_to_element_with_offset(img_tag,x,y).click().perform()
    sleep(1)
print(all_list)
bro.find_element_by_id('username').send_keys('bobo123')
sleep(1)
bro.find_element_by_id('password').send_keys('123456')
sleep(1)
bro.find_element_by_id('loginSub').click()
sleep(2)
bro.quit()


[[194, 66]]


### 12306余票检测

In [7]:
import requests
headers = {
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36',
    'Cookie':'JSESSIONID=05AF8E6620993EA00398CDA88A8943C1; _jc_save_fromStation=%u5317%u4EAC%2CBJP; _jc_save_wfdc_flag=dc; route=6f50b51faa11b987e576cdb301e545c4; BIGipServerpool_passport=216859146.50215.0000; RAIL_EXPIRATION=1585397728408; RAIL_DEVICEID=L0hoksaLWyviy35wtoXKUFX2b2irWMhRkRERdnf3VnzbEAlEx9IqbGRXEUore8HAmGXlAWxzPZa9Kx0_wdcGB2FSMMNJIEP0uBgRy02wQ_l_vMNjQ-k8ZwSA3T5zkSm5LOfTXTX91VPvYa7CcM6AUAiZpw3SoGMO; BIGipServerotn=4057399562.50210.0000; BIGipServerpassport=854065418.50215.0000; _jc_save_fromDate=2020-03-30; _jc_save_toDate=2020-03-26; _jc_save_toStation=%u4E4C%u9C81%u6728%u9F50%u5357%2CWMR'
}
city = {
        '北京北': 'VAP',
        '北京东': 'BOP',
        '北京': 'BJP',
        '北京南': 'VNP',
        '北京西': 'BXP',
        '广州南': 'IZQ',
        '重庆北': 'CUW',
        '重庆': 'CQW',
        '重庆南': 'CRW',
        '广州东': 'GGQ',
        '上海': 'SHH',
        '上海南': 'SNH',
        '上海虹桥': 'AOH'
}

In [8]:
t = input('enter a time:(yyyy-mm-dd):')
start = input('enter a start city:')
start = city[start]
end = input('enter a end city:')
end = city[end]
url = 'https://kyfw.12306.cn/otn/leftTicket/query'
params = {
    'leftTicketDTO.train_date': t,
    'leftTicketDTO.from_station': start,
    'leftTicketDTO.to_station': end,
    'purpose_codes': 'ADULT'
}

json_data_list = requests.get(url=url,headers=headers,params=params).json()['data']['result']
for s in json_data_list:
    print(s)

enter a time:(yyyy-mm-dd):2020-03-28
enter a start city:北京
enter a end city:重庆
h4AuK0a8TH6JcY7%2FBh465yxY1423zWrUh5i0eiMZ3EMbkNZt7AFF5ioQMBBJhBhKZUiuyYyFg%2FQj%0A2sGwOfIGj4FO%2BP3RB8klCE%2BQYxW8qh8Md968%2FqQo46%2BHoAX7d9%2BqwszgtrORN4%2BsC8DMYLRsO433%0AOa%2Be8%2BRdGmqylHGpL0%2FssA6vcCzFEvh6Ypo%2BFLWjGMqF88Oqd8iTbnUW9z4zMJWLfhtuLstwQ2vz%0ArlrvGnFHSB%2BQygieTNmPXowMyQzgDM29CmdoPBGn1SyWbHibh4PzGYyWbjBgpsZ%2FChhTHYpCK683%0App%2BoS2eK1aU%3D|预订|240000K5072K|K507|BXP|GIW|BXP|CXW|07:12|09:35|26:23|Y|jlQxurvTgciGkGiQcPLhPIHldzfN88NqHiBjJn1zWoQE4JtQ9VCw4NmDYVM%3D|20200328|3|PB|01|17|0|0||||18|||无||有|有|||||30401010|3411|0|0|||||||||
T%2FhPvUkqlr6bZ2jpCQ48WlSsqaIrT2DqECM%2BSqT64uD%2Fyw99pyrrrPGYMFobhipId4SvsC%2BT%2FHfK%0A6%2Bx6gcZixIF62bPQiiP2vlVwbpJk9BVSXUJ9Op4c192MI41C%2BsqtYXBkKlWBFIJrJqvpUBHGLXTy%0AKkZl2Gx8plIO32VkN%2BLRVps2Sl5abtWwjRmPG3Rww4WNytHDOP3FRbFqFNlL7JnJDT5PAb8wZHQM%0AyYLVE4u3c5iDOeJCjZzI7ezYhYIKarld3NtjE1QW%2FBOFqqZL%2BLXkEul6wSlSlmg%2F7tXhiSq%2B8nK1%0A|预订|240000G3090J|G309|BXP|CUW|